In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import vmap
from jax import jit, lax
import matplotlib.pyplot as plt
import cmocean as cmo
import importlib

In [2]:
from jax.config import config
config.update("jax_enable_x64", True)

In [22]:
## import modules
import preconditioner as precond
import conjugate_gradient as cg
import pivoted_cholesky as pc
import pivoted_cholesky_ref as pc_ref # to use this script we need "torch", please comment out if not needed.
def reload():
    importlib.reload(precond)
    importlib.reload(cg)
    importlib.reload(pc)
    importlib.reload(pc_ref)
reload()

In [4]:
import gpytorch
import torch
import linear_operator
from linear_operator.operators import (
    AddedDiagLinearOperator,
    DiagLinearOperator,
    LinearOperator,
    DenseLinearOperator,
)

In [5]:
def generate_K(N, seed=0, noise=1e-06):
    """
    generate positive definite symmetric matrix
    """
    K = jax.random.normal(jax.random.PRNGKey(seed), (N, N))
    # K = K @ K.T + 30* jnp.eye(N) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) / N
    K = jnp.dot(K, K.T)/N
    # K += (noise+30)*jnp.eye(N) ## ??
    K += (5)*jnp.eye(N)
    K += (noise)*jnp.eye(N)
    if not is_positive_definite(K):
        raise Exception("K is not positive definite !")
    return K

In [6]:
def is_positive_definite(matrix):
    # 行列の固有値を計算
    eigenvalues = np.linalg.eigvals(matrix)

    # 全ての固有値が正であるかをチェック
    if np.all(eigenvalues > 0):
        return True
    else:
        return False

In [7]:
def rel_error(true, pred):
    nonzero_index = jnp.where(true != 0.)
    true = true[nonzero_index]
    pred = pred[nonzero_index]
    return jnp.mean(jnp.abs((true-pred)/true))

## 4. modified preconditioned conjugate gradient

In [252]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [253]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [254]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [255]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)

In [260]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

2.796436867782459e-09


In [261]:
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

2.475563402797503e-07


### tol=1e-04

In [44]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)
print(rel_error(Kinvy_linalg, Kinvy))

0.0004721970976828854


In [46]:
linear_operator.settings.cg_tolerance._set_value(1e-04)
Kinvy_torch = added_diag.solve(rhs_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
# linear_operator.settings.cg_tolerance._set_value(1)

0.00047221962753508294


In [50]:
N = 7000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [51]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [52]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [53]:
%%timeit
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1e-04)

258 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%%timeit
Kinvy_torch = added_diag.solve(rhs_torch)

1.52 s ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%%timeit
Kinvy_linalg = jnp.linalg.solve(K, rhs)

751 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
print(rel_error(Kinvy_linalg, Kinvy))
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

0.0004721970976828854
0.00047221962753508294


In [57]:
%%timeit
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

156 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### やはりpreconditionしない方がはやいのが現状

## chceking trace term

### with preconditioner

In [86]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [87]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [88]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [89]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))
linear_operator.settings.cg_tolerance._set_value(1)
Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

0.1485656644481834
0.14856548077918746


In [90]:
%%timeit
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1e-04)

73.9 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [91]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [92]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [93]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [99]:
# trace_torch, _ = added_diag.inv_quad_logdet(rhs_torch, logdet=True)

In [98]:
trace, trace_linalg, #trace_torch/2

(DeviceArray(2674.62575194, dtype=float64),
 DeviceArray(2587.2629889, dtype=float64))

In [89]:
(trace_linalg-trace)/trace_linalg

DeviceArray(-0.03376648, dtype=float64)

### without preconditioner

In [100]:
Kinvy = cg.bcg_bbmm(K, rhs, preconditioner=None, print_process=False, tolerance=1)
print(rel_error(Kinvy_linalg, Kinvy))

3448.798220099444


In [101]:
dKdtheta = jax.random.normal(jax.random.PRNGKey(1), (N, N))+5*jnp.eye(N)

In [106]:
trace = jnp.einsum("ij, ij ->", Kinvy[:, :n_tridiag], jnp.einsum("ij, jk->ik", dKdtheta, zs))/n_tridiag

In [107]:
Kinvy_linalg = jnp.linalg.inv(K)
trace_linalg = jnp.sum(jnp.diag(jnp.matmul(Kinvy_linalg, dKdtheta)))

In [108]:
trace, trace_linalg

(DeviceArray(2671.43610977, dtype=float64),
 DeviceArray(2587.2629889, dtype=float64))

In [109]:
(trace_linalg-trace)/trace_linalg

DeviceArray(-0.03253365, dtype=float64)

### checking t_mat

### N = 100

In [23]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [24]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [25]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [26]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

9.241723714046066e-16
7.371856260187996e-06


#### linear_operatorでは, solveの際にn_tridiagをありにするとなぜか精度が向上する

In [27]:
t_mat.shape, t_mat_torch.numpy().shape

((10, 20, 20), (10, 9, 9))

In [28]:
end = 9
jnp.mean(jnp.abs(t_mat[:, :end , :end]-t_mat_torch[:, :end, :end].numpy()))

DeviceArray(0.1816411, dtype=float64)

### N = 3000

In [38]:
N = 3000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [39]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [40]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [41]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

7.393597288661838e-10
2.7001585109192347e-07


In [42]:
t_mat.shape, t_mat_torch.numpy().shape

((10, 20, 20), (10, 20, 20))

In [43]:
end = 7
jnp.mean(jnp.abs(t_mat[:, :end , :end]-t_mat_torch[:, :end, :end].numpy()))

DeviceArray(90717.27715001, dtype=float64)

### Tについてはエラーが大きい: げに一旦sum diagだけを計算するので問題ないのでは?

### checking log determinant

In [104]:
N = 100
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))

In [105]:
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [106]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)
preconditioner_torch, _, _ = added_diag._preconditioner()

In [107]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch,num_tridiag=n_tridiag)
# Kinvy_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))

9.241723714046066e-16
7.371856260187996e-06


In [114]:
inv_quad_torch, logdet_torch = added_diag.inv_quad_logdet(rhs_torch[:, -1:], logdet=True)

In [115]:
inv_quad = jnp.dot(y[:, 0], Kinvy[: ,-1])

In [116]:
inv_quad, inv_quad_torch

(DeviceArray(19.46686596, dtype=float64), tensor(19.4669, dtype=torch.float64))

In [118]:
logdet_torch

tensor(178.0780, dtype=torch.float64)

In [79]:
preconditioner._precond_logdet_cache

DeviceArray(-1146.54738168, dtype=float64)

In [120]:
det_ans = jnp.sum(jnp.log(jnp.diag(K)))
det_ans

DeviceArray(179.3679413, dtype=float64)

In [84]:
e1 = jnp.zeros(t_mat.shape[-2])
e1 = e1.at[0].set(1.)
detK = 0
for i in range(t_mat.shape[-1]):
    eigvals, eigvectors = jnp.linalg.eigh(t_mat[i, :, : ])
    if jnp.any(eigvals<0.):
        # print(eigvals)
        print(i)
        index_nonzero = eigvals >= 0.
        eigvectors = eigvectors.at[:, ~index_nonzero].set(0.)
        eigvals = eigvals.at[~index_nonzero].set(1)
    log_gamma = jnp.diag(jnp.log(eigvals))
    if jnp.any(jnp.isnan(log_gamma)):
        print('NaN !')
        # print(i)
        break
    left_multiplier = jnp.matmul(e1.T, eigvectors)
    detK += jnp.matmul(left_multiplier, jnp.matmul(log_gamma, left_multiplier.T))
    # print(detK)

In [85]:
detK

DeviceArray(35.35987265, dtype=float64)

In [288]:
jnp.log(jnp.linalg.det(K))

DeviceArray(71.12942659, dtype=float64)

In [289]:
jnp.sum(jnp.log(jnp.diag(K)))

DeviceArray(71.63029369, dtype=float64)